In [1]:
import torch
import ltn
import numpy as np
from ltn_imp.automation.knowledge_base import KnowledgeBase
from sklearn.metrics import accuracy_score
import numpy as np
from ltn_imp.automation.data_loaders import LoaderWrapper

## Data and Model Preparation

In [2]:
class ModelA(torch.nn.Module):
    def __init__(self):
        super(ModelA, self).__init__()
        self.elu = torch.nn.ELU()
        self.layer1 = torch.nn.Linear(2, 16)
        self.layer2 = torch.nn.Linear(16, 16)
        self.layer3 = torch.nn.Linear(16, 2) 

    def forward(self, x):
        x = self.elu(self.layer1(x))
        x = self.elu(self.layer2(x))
        logits = self.layer3(x)
        return logits

class LogitsToPredicate(torch.nn.Module):
    def __init__(self, logits_model):
        super(LogitsToPredicate, self).__init__()
        self.logits_model = logits_model
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, y, training=True):
        logits = self.logits_model(x)
        probs = self.softmax(logits)
        # y is expected to be a one-hot encoded vector
        out = torch.sum(probs * y, dim=1, keepdim=True)
        return out

In [3]:
nr_samples = 100
dataset = torch.rand((nr_samples, 2))
labels_dataset = torch.sum(torch.square(dataset - torch.tensor([.5, .5])), dim=1) < .09

In [4]:
class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        n = self.data.shape[0]
        idxlist = list(range(n))
        if self.shuffle:
            np.random.shuffle(idxlist)

        for _, start_idx in enumerate(range(0, n, self.batch_size)):
            end_idx = min(start_idx + self.batch_size, n)
            data = self.data[idxlist[start_idx:end_idx]]
            labels = self.labels[idxlist[start_idx:end_idx]]
            yield data, labels
            
train_loader = DataLoader(dataset[:50], labels_dataset[:50], batch_size=64, shuffle=True)
test_loader = DataLoader(dataset[50:], labels_dataset[50:], batch_size=64, shuffle=False)

In [5]:
def compute_accuracy(loader, model):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = model(data).detach().numpy()
        predictions = np.argmax(predictions, axis=1)
        mean_accuracy += accuracy_score(labels, predictions)
    return mean_accuracy / len(loader)

## My Implementation 

In [6]:
a = ModelA() # type: ignore
model = LogitsToPredicate(a) # type: ignore
predicates = {"Classifier": model}
expression_1 = "all x. (Classifier(x,y))"
rules = [expression_1]

In [7]:
loader = LoaderWrapper(variables=["x"], num_classes=2, target="y", loader=train_loader)

In [8]:
rule_to_data_loader_mapping={expression_1: [loader]} # type: ignore

In [9]:
kb = KnowledgeBase(expressions=rules, predicates=predicates,
                   rule_to_data_loader_mapping=rule_to_data_loader_mapping,
                   quantifier_impls={"forall" : "pmean_error"})

In [10]:
compute_accuracy(model = a, loader= test_loader)

0.7

In [11]:
compute_accuracy(model = a, loader= train_loader)

0.68

In [12]:
kb.optimize(801, log_steps=200)

Epoch 1/801, Loss: 0.517249345779419

Epoch 201/801, Loss: 0.47401976585388184

Epoch 401/801, Loss: 0.4247232675552368

Epoch 601/801, Loss: 0.27480655908584595

Epoch 801/801, Loss: 0.16216826438903809



In [13]:
compute_accuracy(model = a, loader= train_loader)

0.98

In [14]:
compute_accuracy(model = a, loader= test_loader)

1.0

## LTN

In [15]:
class ModelA(torch.nn.Module):
    def __init__(self):
        super(ModelA, self).__init__()
        self.sigmoid = torch.nn.Sigmoid()
        self.layer1 = torch.nn.Linear(2, 16)
        self.layer2 = torch.nn.Linear(16, 16)
        self.layer3 = torch.nn.Linear(16, 1)
        self.elu = torch.nn.ELU() 

    def forward(self, x):
        x = self.elu(self.layer1(x))
        x = self.elu(self.layer2(x))
        return self.sigmoid(self.layer3(x)) 

In [16]:
A = ltn.Predicate(ModelA()) # type: ignore
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()

In [17]:
def compute_sat_level(loader):
    mean_sat = 0
    for data, labels in loader:
        
        x_A = ltn.Variable("x_A", data[torch.nonzero(labels)])  # positive examples
        x_not_A = ltn.Variable("x_not_A", data[torch.nonzero(torch.logical_not(labels))])  # negative examples

        mean_sat += SatAgg(
            Forall(x_A, A(x_A)),
            Forall(x_not_A, Not(A(x_not_A)))
        )
        
    mean_sat /= len(loader)
    return mean_sat

def compute_accuracy(loader):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = A.model(data).detach().numpy()
        predictions = np.where(predictions > 0.5, 1., 0.).flatten()
        mean_accuracy += accuracy_score(labels, predictions)

    return mean_accuracy / len(loader)

In [18]:
optimizer = torch.optim.Adam(A.parameters(), lr=0.001)


for epoch in range(801):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        x_A = ltn.Variable("x_A", data[torch.nonzero(labels)]) # positive examples
        x_not_A = ltn.Variable("x_not_A", data[torch.nonzero(torch.logical_not(labels))]) # negative examples

        sat_agg = SatAgg(
            Forall(x_A, A(x_A)),
            Forall(x_not_A, Not(A(x_not_A)))
        )

        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss = train_loss / len(train_loader)

    if epoch % 200 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
        %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
            compute_accuracy(train_loader), compute_accuracy(test_loader)))
        
        print()
        print(f"Positive { Forall(x_A, A(x_A)) }")
        print(f"Negative { Forall(x_not_A, Not(A(x_not_A)))}") 
        print()

 epoch 0 | loss 0.5045 | Train Sat 0.496 | Test Sat 0.499 | Train Acc 0.320 | Test Acc 0.300

Positive LTNObject(value=tensor(0.5481, grad_fn=<RsubBackward1>), free_vars=[])
Negative LTNObject(value=tensor(0.4492, grad_fn=<RsubBackward1>), free_vars=[])

 epoch 200 | loss 0.4780 | Train Sat 0.522 | Test Sat 0.486 | Train Acc 0.620 | Test Acc 0.500

Positive LTNObject(value=tensor(0.5500, grad_fn=<RsubBackward1>), free_vars=[])
Negative LTNObject(value=tensor(0.4957, grad_fn=<RsubBackward1>), free_vars=[])

 epoch 400 | loss 0.4394 | Train Sat 0.561 | Test Sat 0.549 | Train Acc 0.680 | Test Acc 0.680

Positive LTNObject(value=tensor(0.5879, grad_fn=<RsubBackward1>), free_vars=[])
Negative LTNObject(value=tensor(0.5359, grad_fn=<RsubBackward1>), free_vars=[])

 epoch 600 | loss 0.2781 | Train Sat 0.723 | Test Sat 0.730 | Train Acc 0.920 | Test Acc 0.980

Positive LTNObject(value=tensor(0.7429, grad_fn=<RsubBackward1>), free_vars=[])
Negative LTNObject(value=tensor(0.7039, grad_fn=<RsubBa